- **Author:** Dace Apšvalka, MRC CBU
- **Date:** February 2026
- **Requirements:** MNE, meg_event_utils repository

# Utility functions for cleaning, recovering, plotting, and QA of MEG trigger events

In most MEG experiments, the stimulus delivery program (e.g. PsychoPy, Presentation, E-Prime) sends event triggers to the MEG system, which are recorded on one or more trigger channels. These triggers are the backbone of epoching and condition labelling: they tell us what happened when.

Depending on the acquisition setup, triggers may be saved either as multiple separate “bit” channels (each representing one digital line) or as a single summed channel (often called STI101 or similar), which encodes the state of all bits into one integer value.

In practice, however, trigger recordings are often messy. You may encounter spurious pulses, overlapping triggers (e.g. stimulus and keypress at the same time), or incomplete recordings where only a summed trigger channel was saved instead of the individual bit channels. These issues can lead to missing events, extra non-existent events, or mislabelled trials if not handled carefully.

The package `meg_event_utils` provides a small collection of helper functions to clean, recover, inspect, and quality-check MEG trigger events, with a focus on making these problems easier to diagnose and fix.

Libraries such as MNE-Python already provide excellent tools for reading and extracting events from trigger channels in standard cases. However, in real-world datasets with spurious triggers, overlapping codes, or incomplete trigger recordings, additional inspection and custom handling are often needed. The functions in `meg_event_utils` are designed to complement existing workflows by making these edge cases easier to diagnose and fix.

Below, I describe the typical problems and their solutions, followed by a worked example showing a typical workflow for inspecting and extracting events.

## Problems

### Spurious triggers, creating non-existent stimulus events

### Overlapping keypress + stimulus triggers

### Only the summed STI101 saved, but no separate (or not all separate) channels

## Solutions

Assumes, only STI101 available. Decomposes STI101 into individual channels. 

## Usual use case - get your events

What is it doing differently than MNE? 

## Check your triggers/events